# Third Method Neural Network Implementations

<p><b>Author</b>: Jingze Dai</p>
<p><b>McMaster University</b>, Honors Computer Science (Coop) student</p>
<p><b>Personal Email Address</b>: <a>david1147062956@gmail.com</a>, or <a>dai.jingze@icloud.com</a></p>
<a href="https://github.com/daijingz">Github Homepage</a>
<a href="https://www.linkedin.com/in/jingze-dai/">Linkedin Webpage</a>
<a href="https://leetcode.com/david1147062956/">Leetcode Webpage</a>

<i>The original research's third featuring method with its corresponding ANN, CNN, and LSTM implementations.</i>

<i>Your Feedback is important for Jingze's further development. If you want to give feedback and suggestions, or you want to participate in working and learning together, please email Jingze at dai.jingze@icloud.com. If you want Jingze to provide contributions to your research or opensource project or you want Jingze to help you with any programming issues, please email Jingze at david1147062956@gmail.com. Thank you for your help.</i>

## Table of Contents
* [Section 1: Selected Features](#bullet1)
* [Section 2: Extract and Load Datasets](#bullet2)
* [Section 3: Classification (Binary Classification Approach (BCA))](#bullet3)
* [Section 4: Classification (A Multi-class Classification Approach for Three Classes (MCATC))](#bullet4)
* [Section 5: Classification (A Classic Learning Approach for Multi-class classification (C-LAMC))](#bullet5)
* [Section 6: Improvement Works on C-LAMC Classifications](#bullet6)

### <a class="anchor" id="bullet1"><p><b>Section 1</b>: Selected Features</p></a>

There are 12 features selected: `posx`, `posy`, `spdx`, `spdy`, `spdx_n`, `spdy_n`, `aclx`, `acly`, `hedx`, `hedy`,`hedx_n`, and `hedy_n`

### <a class="anchor" id="bullet2"><p><b>Section 2</b>: Extract and Load Datasets</p></a>

In [2]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade tensorflow --user

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --user numpy==1.24.4

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --upgrade scipy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import gdown

# Load the dataset
output_file = 'mixalldata_clean.csv'
df = pd.read_csv(output_file)

# Display the DataFrame
print(df.head())

   type      sendTime  sender  senderPseudo  messageID  class        posx  \
0     4  72002.302942  130137     101301377  422013806      0  266.982401   
1     4  72003.302942  130137     101301377  422023410      0  266.827208   
2     4  72004.302942  130137     101301377  422032081      0  266.420297   
3     4  72005.302942  130137     101301377  422040712      0  268.912026   
4     4  72006.302942  130137     101301377  422052949      0  268.242276   

        posy  posz    posx_n  ...  aclz    aclx_n    acly_n  aclz_n      hedx  \
0  32.336955   0.0  3.480882  ...   0.0  0.000862  0.000862     0.0 -0.102790   
1  34.624145   0.0  3.546261  ...   0.0  0.000107  0.001040     0.0 -0.099856   
2  38.836461   0.0  3.544045  ...   0.0  0.000172  0.001661     0.0 -0.099856   
3  45.414229   0.0  3.340080  ...   0.0  0.000171  0.001654     0.0 -0.100172   
4  53.729986   0.0  3.328872  ...   0.0  0.000193  0.001852     0.0 -0.097105   

       hedy  hedz     hedx_n     hedy_n  hedz_n  


### <a class="anchor" id="bullet3"><p><b>Section 3</b>: Classification (Binary Classification Approach (BCA))</p></a>

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tffrom tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features]
Y = (df['class'] != 0).astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(64, input_shape=(len(features),), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

print("ANN Accuracy:", accuracy)
print("ANN Precision:", precision)
print("ANN Recall:", recall)
print("ANN F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 74s 1ms/step - accuracy: 0.6902 - loss: 0.6245 - val_accuracy: 0.7628 - val_loss: 0.4963
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 15s 769us/step
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 15s 760us/step - accuracy: 0.7614 - loss: 0.4966
ANN Accuracy: 0.7618277668952942
ANN Precision: 0.9234229960308048
ANN Recall: 0.4504236618110434
ANN F1-score: 0.6054992599044486


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import precision_score, recall_score, f1_score

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features].values
Y = (df['class'] != 0).astype(int).values

sequence_length = len(features)
X = X.reshape(-1, sequence_length, 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred = model.predict(X_test)
Y_pred_binary = (Y_pred > 0.5).astype(int)

accuracy = accuracy_score(Y_test, Y_pred_binary)
precision = precision_score(Y_test, Y_pred_binary)
recall = recall_score(Y_test, Y_pred_binary)
f1 = f1_score(Y_test, Y_pred_binary)

print("CNN Accuracy:", accuracy)
print("CNN Precision:", precision)
print("CNN Recall:", recall)
print("CNN F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 135s 2ms/step - accuracy: 0.7339 - loss: 0.5728 - val_accuracy: 0.7898 - val_loss: 0.4625
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 20s 998us/step
CNN Accuracy: 0.7885555009531083
CNN Precision: 0.9614509943815215
CNN Recall: 0.498941327563665
CNN F1-score: 0.6569572139151896


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import precision_score, recall_score, f1_score

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features].values
Y = (df['class'] != 0).astype(int).values

sequence_length = len(features)
X = X.reshape(-1, sequence_length, 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    LSTM(64, input_shape=(sequence_length, 1)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

print("LSTM Accuracy:", accuracy)
print("LSTM Precision:", precision)
print("LSTM Recall:", recall)
print("LSTM F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 286s 4ms/step - accuracy: 0.7590 - loss: 0.5038 - val_accuracy: 0.7914 - val_loss: 0.4638
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.7902 - loss: 0.4651
LSTM Accuracy: 0.7911252975463867
LSTM Precision: 0.9606103203889181
LSTM Recall: 0.5060184274568336
LSTM F1-score: 0.662862397094003


### <a class="anchor" id="bullet4"><p><b>Section 4</b>: Classification (A Multi-class Classification Approach for Three Classes (MCATC))</p></a>

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def characterize_class(value):
    if value == 0:
        return 0
    elif value >= 1 and value <= 12:
        return 1
    else:
        return 2

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features].values
Y = df['class'].apply(characterize_class).astype(int).values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(64, input_shape=(len(features),), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

print("ANN Accuracy:", accuracy)
print("ANN Precision:", precision)
print("ANN Recall:", recall)
print("ANN F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 223s 3ms/step - accuracy: 0.3344 - loss: -4652460.0000 - val_accuracy: 0.3753 - val_loss: -68858608.0000
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 49s 2ms/step
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 45s 2ms/step - accuracy: 0.3735 - loss: -69851048.0000


C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ANN Accuracy: 0.37397685647010803
ANN Precision: 0.43353787181448067
ANN Recall: 0.37397685621367155
ANN F1-score: 0.3640409172827655


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define the function to characterize class
def characterize_class(value):
    if value == 0:
        return 0
    elif value >= 1 and value <= 12:
        return 1
    else:
        return 2

# List of features
features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']

# Preparing data
X = df[features].values
Y = df['class'].apply(characterize_class).astype(int).values
Y = to_categorical(Y)  # Convert to one-hot encoding

# Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Normalizing the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Building the model
model = Sequential([
    Dense(128, input_shape=(len(features),), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # Adjusted for multi-class classification
])

# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping and learning rate reduction on plateau
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)

# Training the model
model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

# Evaluating the model
Y_pred_prob = model.predict(X_test)
Y_pred = np.argmax(Y_pred_prob, axis=1)
Y_test_classes = np.argmax(Y_test, axis=1)

# Computing metrics
accuracy = accuracy_score(Y_test_classes, Y_pred)
precision = precision_score(Y_test_classes, Y_pred, average='weighted')
recall = recall_score(Y_test_classes, Y_pred, average='weighted')
f1 = f1_score(Y_test_classes, Y_pred, average='weighted')

# Printing the results
print("ANN Accuracy:", accuracy)
print("ANN Precision:", precision)
print("ANN Recall:", recall)
print("ANN F1-score:", f1)

Epoch 1/50


C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 84s 1ms/step - accuracy: 0.7195 - loss: 0.7487 - val_accuracy: 0.7660 - val_loss: 0.6638 - learning_rate: 0.0010
Epoch 2/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 84s 1ms/step - accuracy: 0.7539 - loss: 0.6862 - val_accuracy: 0.7732 - val_loss: 0.6497 - learning_rate: 0.0010
Epoch 3/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 85s 1ms/step - accuracy: 0.7594 - loss: 0.6762 - val_accuracy: 0.7758 - val_loss: 0.6430 - learning_rate: 0.0010
Epoch 4/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 83s 1ms/step - accuracy: 0.7624 - loss: 0.6705 - val_accuracy: 0.7794 - val_loss: 0.6353 - learning_rate: 0.0010
Epoch 5/50
62919/63897 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7640 - loss: 0.6674

KeyboardInterrupt: 

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import precision_score, recall_score, f1_score

def characterize_class(value):
    if value == 0:
        return 0
    elif value >= 1 and value <= 12:
        return 1
    else:
        return 2

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features].values
Y = df['class'].apply(characterize_class).astype(int).values

sequence_length = len(features)
X = X.reshape(-1, sequence_length, 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred = model.predict(X_test)
Y_pred_binary = (Y_pred > 0.5).astype(int)

accuracy = accuracy_score(Y_test, Y_pred_binary)
precision = precision_score(Y_test, Y_pred_binary, average='weighted')
recall = recall_score(Y_test, Y_pred_binary, average='weighted')
f1 = f1_score(Y_test, Y_pred_binary, average='weighted')

print("CNN Accuracy:", accuracy)
print("CNN Precision:", precision)
print("CNN Recall:", recall)
print("CNN F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 295s 5ms/step - accuracy: 0.3406 - loss: -4908548.5000 - val_accuracy: 0.3646 - val_loss: -70431504.0000
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 23s 1ms/step


C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CNN Accuracy: 0.3637462008695353
CNN Precision: 0.42212103826182124
CNN Recall: 0.3637462008695353
CNN F1-score: 0.354193808658255


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import precision_score, recall_score, f1_score

def characterize_class(value):
    if value == 0:
        return 0
    elif value >= 1 and value <= 12:
        return 1
    else:
        return 2

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features].values
Y = df['class'].apply(characterize_class).astype(int).values

sequence_length = len(features)
X = X.reshape(-1, sequence_length, 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    LSTM(64, input_shape=(sequence_length, 1)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

print("LSTM Accuracy:", accuracy)
print("LSTM Precision:", precision)
print("LSTM Recall:", recall)
print("LSTM F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 836s 13ms/step - accuracy: 0.4560 - loss: -70.9306 - val_accuracy: 0.3963 - val_loss: -326.7960
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - accuracy: 0.3959 - loss: -327.3655


C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


LSTM Accuracy: 0.3964727222919464
LSTM Precision: 0.46413267372450967
LSTM Recall: 0.3964727166873773
LSTM F1-score: 0.38950174668682536


### <a class="anchor" id="bullet5"><p><b>Section 5</b>: Classification (A Classic Learning Approach for Multi-class classification (C-LAMC))</p></a>

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features]
Y = df['class'].astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(64, input_shape=(len(features),), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

print("ANN Accuracy:", accuracy)
print("ANN Precision:", precision)
print("ANN Recall:", recall)
print("ANN F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 79s 1ms/step - accuracy: 0.0138 - loss: -116202307584.0000 - val_accuracy: 0.0137 - val_loss: -1664064421888.0000
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 15s 753us/step
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 16s 812us/step - accuracy: 0.0136 - loss: -1679659106304.0000


C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ANN Accuracy: 0.013424898497760296
ANN Precision: 0.00018022789993040557
ANN Recall: 0.013424898507266473
ANN F1-score: 0.0003556808209387275


Here is an early-stage improved version of the ANN network above, which adds more layers. The accuracy has been increased a lot without considering the instability problem. -> That is, abnormal values of the statistics `loss` and `val_loss`.

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features]
Y = df['class'].astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(256, input_shape=(len(features),), activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)  # Increased epochs for more training

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

print("ANN Accuracy:", accuracy)
print("ANN Precision:", precision)
print("ANN Recall:", recall)
print("ANN F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 125s 2ms/step - accuracy: 0.0138 - loss: -3792197187379658752.0000 - val_accuracy: 0.0137 - val_loss: -109648269314380267520.0000
Epoch 2/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 114s 2ms/step - accuracy: 0.0137 - loss: -390503464514598469632.0000 - val_accuracy: 0.0137 - val_loss: -2584283122725455659008.0000
Epoch 3/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 121s 2ms/step - accuracy: 0.0137 - loss: -4888723260655223701504.0000 - val_accuracy: 0.0137 - val_loss: -17266669241049381076992.0000
Epoch 4/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 113s 2ms/step - accuracy: 0.2669 - loss: nan - val_accuracy: 0.5952 - val_loss: nan
Epoch 5/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 114s 2ms/step - accuracy: 0.5946 - loss: nan - val_accuracy: 0.5952 - val_loss: nan
Epoch 6/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 113s 2ms/step - accuracy: 0.5953 - loss: nan - val_accuracy: 0.5952 - val_loss: nan
Epoch 7/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 122s 2ms/step - accuracy: 0.5950 - loss: n

C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ANN Accuracy: 0.594205915927887
ANN Precision: 0.35308066968094687
ANN Recall: 0.5942059152187454
ANN F1-score: 0.44295491104422313


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import precision_score, recall_score, f1_score

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features].values
Y = df['class'].astype(int).values

sequence_length = len(features)
X = X.reshape(-1, sequence_length, 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred = model.predict(X_test)
Y_pred_binary = (Y_pred > 0.5).astype(int)

accuracy = accuracy_score(Y_test, Y_pred_binary)
precision = precision_score(Y_test, Y_pred_binary, average='weighted')
recall = recall_score(Y_test, Y_pred_binary, average='weighted')
f1 = f1_score(Y_test, Y_pred_binary, average='weighted')

print("CNN Accuracy:", accuracy)
print("CNN Precision:", precision)
print("CNN Recall:", recall)
print("CNN F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 132s 2ms/step - accuracy: 0.0144 - loss: -190857625600.0000 - val_accuracy: 0.0137 - val_loss: -2633349726208.0000
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step


C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CNN Accuracy: 0.013424898507266473
CNN Precision: 0.00018022789993040557
CNN Recall: 0.013424898507266473
CNN F1-score: 0.0003556808209387275


In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import precision_score, recall_score, f1_score

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features].values
Y = df['class'].astype(int).values

sequence_length = len(features)
X = X.reshape(-1, sequence_length, 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    LSTM(64, input_shape=(sequence_length, 1)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

print("LSTM Accuracy:", accuracy)
print("LSTM Precision:", precision)
print("LSTM Recall:", recall)
print("LSTM F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 295s 5ms/step - accuracy: 0.0141 - loss: -3793.4465 - val_accuracy: 0.0137 - val_loss: -15081.2393
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.0136 - loss: -15181.0928


C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


LSTM Accuracy: 0.013424898497760296
LSTM Precision: 0.00018022789993040557
LSTM Recall: 0.013424898507266473
LSTM F1-score: 0.0003556808209387275


Unlike previous version, this time we can see that almost all neural networks have severe abnormal `loss` and `val_loss` values. In this research, the normal `val_loss` value is a decimal number between 0 and 1. (Like the first section.)

Obviously, these extremely large negative values for `loss` and `val_loss` indicate a severe numerical instability problem. (Bad news) Simultaneously, I observe that by inserting new neural network layers, the classification accuracy can be considerably improved. (Good news)

### <a class="anchor" id="bullet6"><p><b>Section 6</b>: Improvement Works on Previous Results from Classification (A Multi-class Classification Approach for Three Classes (MCATC))</p></a>

Now I applied several different techniques inside the existing code:
1. Dropout Layers: Help prevent overfitting by randomly setting a fraction of input units to 0 at each update during training.
2. ReduceLROnPlateau: Reduces learning rate when a metric has stopped improving to help the model converge better.
3. Early Stopping: Monitors validation loss and stops training when improvement ceases to prevent overfitting.

Simultaneouly, I inserted several layers for my ANN implementation, with an improvement of overall accuracy statistics:

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def characterize_class(value):
    if value == 0:
        return 0
    elif value >= 1 and value <= 12:
        return 1
    else:
        return 2

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']

X = df[features].values
Y = df['class'].apply(characterize_class).astype(int).values
Y = to_categorical(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Normalizing the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(128, input_shape=(len(features),), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)

model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

Y_pred_prob = model.predict(X_test)
Y_pred = np.argmax(Y_pred_prob, axis=1)
Y_test_classes = np.argmax(Y_test, axis=1)

accuracy = accuracy_score(Y_test_classes, Y_pred)
precision = precision_score(Y_test_classes, Y_pred, average='weighted')
recall = recall_score(Y_test_classes, Y_pred, average='weighted')
f1 = f1_score(Y_test_classes, Y_pred, average='weighted')

print("ANN Accuracy:", accuracy)
print("ANN Precision:", precision)
print("ANN Recall:", recall)
print("ANN F1-score:", f1)

Epoch 1/50


C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 84s 1ms/step - accuracy: 0.7192 - loss: 0.7483 - val_accuracy: 0.7664 - val_loss: 0.6641 - learning_rate: 0.0010
Epoch 2/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 84s 1ms/step - accuracy: 0.7537 - loss: 0.6867 - val_accuracy: 0.7744 - val_loss: 0.6490 - learning_rate: 0.0010
Epoch 3/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 87s 1ms/step - accuracy: 0.7589 - loss: 0.6774 - val_accuracy: 0.7759 - val_loss: 0.6429 - learning_rate: 0.0010
Epoch 4/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 84s 1ms/step - accuracy: 0.7615 - loss: 0.6722 - val_accuracy: 0.7777 - val_loss: 0.6400 - learning_rate: 0.0010
Epoch 5/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 84s 1ms/step - accuracy: 0.7643 - loss: 0.6672 - val_accuracy: 0.7794 - val_loss: 0.6373 - learning_rate: 0.0010
Epoch 6/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 88s 1ms/step - accuracy: 0.7659 - loss: 0.6646 - val_accuracy: 0.7810 - val_loss: 0.6344 - learning_rate: 0.0010
Epoch 7/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 84s 1ms/step - accuracy: 

But there are still some distances between this result and the expected 90% goal. In the next step, I added a few more neural network layers inside my ANN model.

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def characterize_class(value):
    if value == 0:
        return 0
    elif value >= 1 and value <= 12:
        return 1
    else:
        return 2

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']

X = df[features].values
Y = df['class'].apply(characterize_class).astype(int).values
Y = to_categorical(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Normalizing the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(256, input_shape=(len(features),), activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)

model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

Y_pred_prob = model.predict(X_test)
Y_pred = np.argmax(Y_pred_prob, axis=1)
Y_test_classes = np.argmax(Y_test, axis=1)

accuracy = accuracy_score(Y_test_classes, Y_pred)
precision = precision_score(Y_test_classes, Y_pred, average='weighted')
recall = recall_score(Y_test_classes, Y_pred, average='weighted')
f1 = f1_score(Y_test_classes, Y_pred, average='weighted')

print("ANN Accuracy:", accuracy)
print("ANN Precision:", precision)
print("ANN Recall:", recall)
print("ANN F1-score:", f1)

Epoch 1/50


C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 107s 2ms/step - accuracy: 0.7234 - loss: 0.7435 - val_accuracy: 0.7731 - val_loss: 0.6507 - learning_rate: 0.0010
Epoch 2/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 113s 2ms/step - accuracy: 0.7633 - loss: 0.6720 - val_accuracy: 0.7790 - val_loss: 0.6396 - learning_rate: 0.0010
Epoch 3/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 107s 2ms/step - accuracy: 0.7689 - loss: 0.6609 - val_accuracy: 0.7831 - val_loss: 0.6315 - learning_rate: 0.0010
Epoch 4/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 103s 2ms/step - accuracy: 0.7716 - loss: 0.6558 - val_accuracy: 0.7845 - val_loss: 0.6277 - learning_rate: 0.0010
Epoch 5/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 107s 2ms/step - accuracy: 0.7734 - loss: 0.6525 - val_accuracy: 0.7858 - val_loss: 0.6237 - learning_rate: 0.0010
Epoch 6/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 113s 2ms/step - accuracy: 0.7749 - loss: 0.6495 - val_accuracy: 0.7845 - val_loss: 0.6272 - learning_rate: 0.0010
Epoch 7/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 107s 2ms/step - acc

Seems like adding layers may not improved ANN accuracy obviously. Then I tried to adjust the numbers of neurons and the dropout rate, in order to find the optimal accuracy. There are five values in the `Dropout()` function: 0.1, 0.3, 0.5, 0.7, and 0.9.

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def characterize_class(value):
    if value == 0:
        return 0
    elif value >= 1 and value <= 12:
        return 1
    else:
        return 2

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']

X = df[features].values
Y = df['class'].apply(characterize_class).astype(int).values
Y = to_categorical(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Normalizing the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(256, input_shape=(len(features),), activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)

model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

Y_pred_prob = model.predict(X_test)
Y_pred = np.argmax(Y_pred_prob, axis=1)
Y_test_classes = np.argmax(Y_test, axis=1)

accuracy = accuracy_score(Y_test_classes, Y_pred)
precision = precision_score(Y_test_classes, Y_pred, average='weighted')
recall = recall_score(Y_test_classes, Y_pred, average='weighted')
f1 = f1_score(Y_test_classes, Y_pred, average='weighted')

print("ANN Accuracy:", accuracy)
print("ANN Precision:", precision)
print("ANN Recall:", recall)
print("ANN F1-score:", f1)

Epoch 1/50


C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 104s 2ms/step - accuracy: 0.7441 - loss: 0.7046 - val_accuracy: 0.7831 - val_loss: 0.6283 - learning_rate: 0.0010
Epoch 2/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 107s 2ms/step - accuracy: 0.7762 - loss: 0.6427 - val_accuracy: 0.7890 - val_loss: 0.6184 - learning_rate: 0.0010
Epoch 3/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 103s 2ms/step - accuracy: 0.7827 - loss: 0.6284 - val_accuracy: 0.7947 - val_loss: 0.6022 - learning_rate: 0.0010
Epoch 4/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 110s 2ms/step - accuracy: 0.7864 - loss: 0.6205 - val_accuracy: 0.7957 - val_loss: 0.5991 - learning_rate: 0.0010
Epoch 5/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 114s 2ms/step - accuracy: 0.7883 - loss: 0.6165 - val_accuracy: 0.7986 - val_loss: 0.5925 - learning_rate: 0.0010
Epoch 6/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 106s 2ms/step - accuracy: 0.7897 - loss: 0.6141 - val_accuracy: 0.7996 - val_loss: 0.5907 - learning_rate: 0.0010
Epoch 7/50
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 104s 2ms/step - acc

From training we can see that although the overall accuracy is high, most of these training steps take longer time and much more steps.